In [1]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD   
import random

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)


for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
#training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = [item[0] for item in training]
train_y = [item[1] for item in training]
print("Training data created")


# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=300, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")


[nltk_data] Downloading package punkt to C:\Users\SPIRO-
[nltk_data]     PYTHON1\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\SPIRO-
[nltk_data]     PYTHON1\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


210 documents
48 classes [' Facilities & Campus Life', ' pro_chancellor', 'Aerospace Engineering_HOD,Head_of_the_Department', 'Automobile Engineering_HOD,Head_of_the_Department', 'CA_computer_applicatio_Faculty,teaching_Staff,Professors', 'CA_computer_application_HOD,Head_of_the_Department', 'CSE_HOD,Head_of_the_Department', 'Civil Engineering_HOD,Head_of_the_Department', 'Contact_info', 'ECE_HOD,Head_of_the_Department', 'EEE_HOD,Head_of_the_Department', 'EIE_HOD,Head_of_the_Department', 'English_HOD,Head_of_the_Department', 'IT,Information_Technology,HOD,Head_of_the_Department', 'Introduction', 'LAW_HOD,Head_of_the_Department', 'MAC,mathematics&actuarial_science_HOD,Head_of_the_Department', 'MBA,Mnagement_studies_HOD,Head_of_the_Department', 'Mechanical Engineering_HOD,Head_of_the_Department', 'Polymer Engineering_HOD,Head_of_the_Department', 'arabic_and_islamic_studies_HOD,Head_of_the_Department', 'chancellor', 'chemistry_HOD,Head_of_the_Department', 'commerce_HOD,Head_of_the_Departm

F:\SOFTWARE\ANACONDA\envs\face\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


42/42 [==============================] - 1s 2ms/step - loss: 3.9231 - accuracy: 0.0095
Epoch 2/300
42/42 [==============================] - 0s 2ms/step - loss: 3.8779 - accuracy: 0.0095
Epoch 3/300
42/42 [==============================] - 0s 2ms/step - loss: 3.7737 - accuracy: 0.0667
Epoch 4/300
42/42 [==============================] - 0s 2ms/step - loss: 3.7357 - accuracy: 0.0714
Epoch 5/300
42/42 [==============================] - 0s 2ms/step - loss: 3.5744 - accuracy: 0.1190
Epoch 6/300
42/42 [==============================] - 0s 2ms/step - loss: 3.4161 - accuracy: 0.1286
Epoch 7/300
42/42 [==============================] - 0s 3ms/step - loss: 3.2815 - accuracy: 0.1381
Epoch 8/300
42/42 [==============================] - 0s 2ms/step - loss: 3.1529 - accuracy: 0.1619
Epoch 9/300
42/42 [==============================] - 0s 2ms/step - loss: 2.9182 - accuracy: 0.1857
Epoch 10/300
42/42 [==============================] - 0s 2ms/step - loss: 2.7749 - accuracy: 0.2667
Epoch 11/300
42/42 [=

In [2]:
# !pip freeze > requirements.txt

In [3]:
# !python --version

In [4]:
#! pip install -r requirements.txt
# pip install --upgrade googletrans==4.0.0-rc1
# pip install translate